In [136]:
import sys
sys.path.append(r"D:\Scripts1\Code\ActPEA\CODE\Actu_Colums.py")
import pandas as pd
import numpy as np
import Actu_Colums as Ac
import win32com.client as win32
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import datetime
import os,time
import glob
from typing import Optional
from babel.dates import format_date, format_datetime, Locale
import re
import pickle
import subprocess



### El flujo de actualizacion es el siguiente : 
    -01.Extraigo un fragmente del prepa_global_N solo de EI
    -01,5. Concateno los Bloqueos de Factura
    -02.Hereda informacion del Prepa_old
    -03.Actualizo info del PAP
    -03,5. Añado analistas
    -04.Vuelvo a meter el fragmento en el prepa_global_N
    -05.To Excel

## Time pointer and funcions

In [137]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
Today_date = datetime.date.today()
filas_in = Today_date
Today_str = datetime.date.today().strftime('%d-%m-%Y')
Today_D_M = Today_str[0:5]
Today_D_M = [Today_D_M[0:2],'.',Today_D_M[3:6]]
Today_D_M = ''.join(Today_D_M)
with open(r"\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Dimensiones\temp.pkl", "rb") as f:
    filas_out = pickle.load(f)
print('Hoy es:',Today_D_M)
def process_df_V1(df: pd.DataFrame,clases_unicas: list,index=['NOMPROVEEDOR','ESTADO']):
    """ La funcion recibe un df, y las clases unicas
        Y devuelve el df uniformizado para cada clase
        Llena los valores faltantes con 0"""
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #Agrupo 
    grupos = pivot_table.groupby('NOMPROVEEDOR') #creo un data frame para cada contrata
    # Para cada grupo, crea un nuevo dataframe y guárdalo en un diccionario
    dataframes = {}
    for nombre, datos in grupos: # Creo los df dentro del dic
        dataframes[nombre] = datos
    for key in dataframes.keys(): # para cada df en el dic
        df = dataframes[key].reset_index()
        df1 = df.set_index('ESTADO').reindex(clases_unicas).reset_index() #normalizo, creando filas para todos los ESTADOS
        df1.NOMPROVEEDOR = key  #Relleno los NAN
        df1 = df1.fillna(0)  
        dataframes[key] = df1   #Reescribo los Df para cada clave
    df_concat = pd.concat(dataframes.values(),ignore_index=True) # Compacto todo los df del dic en uno grande 
    pivot_table_1 = pd.pivot_table(df_concat, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #agrupo de nuevo
    return pivot_table_1
################################################################################################3

def calc_diff_V1(df1: pd.DataFrame,df2: pd.DataFrame, index= ['NOMPROVEEDOR', 'ESTADO']) -> pd.DataFrame:
    """ entran 2 dataframes , los proceso , y en base a los df procesados 
        calculo la diferencia en una nueva columna, 
        y añado esta columna de diferencia al df actual """
    clases_unicas = pd.concat([df1['ESTADO'], df2['ESTADO']]).unique() #Creo listas de claves unicas
    df1_proces = process_df_V1(df1,clases_unicas)
    df2_proces = process_df_V1(df2,clases_unicas)
    
    df_diff = df1_proces - df2_proces
    diff_pivot_table_reset = df_diff.reset_index()
    diff_pivot_table_reset.rename(columns={'PEND_FACT_SOLES': 'DIFERENCIA EN SOLES'}, inplace=True)
    # Merge diff_pivot_table_filled_reset con PrePa_O_EI
    PrePa_O_EI = pd.merge(df1, diff_pivot_table_reset, on=index, how='left')    
    return PrePa_O_EI

def filtro(df):
    return df[df.RESPONSABLE2 == 'Eduardo Iberico'] #Filtro 
    
def get_2_df_from_path(path: str,extension: str, Week_Int: int,sheet_name: str) -> list[pd.DataFrame]:
    """ Extrae los archivos mas recientes de un path"""
    path= path+'/*'
    tipo_de_archivo = '*'+ extension
    archivos = glob.glob(path + tipo_de_archivo)
    archivos_ordenados = sorted(archivos, key=os.path.getctime, reverse=True)
    nombres_de_archivos = [os.path.basename(archivo) for archivo in archivos_ordenados[:1] + archivos_ordenados[Week_Int-1:Week_Int]] 
    print(nombres_de_archivos)
    dataframes = []  # Lista para almacenar los DataFrames
    for nombre in nombres_de_archivos:
        path_of_e = path[:-1] + '{}'.format(nombre)
        df = pd.read_excel(path_of_e, sheet_name=sheet_name, dtype={'COMENTARIO': str})
        dataframes.append(df)
    return dataframes
        
def convert_to_date(val):
    try:
        return pd.Timestamp('1899-12-30') + pd.Timedelta(int(float(val)), 'D')
    except ValueError:
        return val

def get_recent_df(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente modificado de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getmtime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,archivo_mas_reciente
def get_recent_c_df(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente creado de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N
def cleanrows(df):
    if 'STATUSFINAL' in df.columns:
        df = df[df['STATUSFINAL'] == 'PENDIENTE']
    elif 'STATUS FINAL' in df.columns:
        df = df[df['STATUS FINAL'] == 'PENDIENTE']
    return df

def process_to_bcsv(df : pd.DataFrame,ruta_del_csv : str,Fecha: str):

    df['TIME'] = Fecha
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=['TIME','MES-COMPROMISO','NOMPROVEEDOR','ESTADO'], aggfunc=pd.Series.sum) #Agrupo 
    df_reset = pivot_table.reset_index(drop=False)
    #convetir la columna del agrupado al formato d efehca 
    # Convierte la columna 'Fecha' a datetime
    df_reset['TIME'] = pd.to_datetime(df_reset['TIME'],format = '%d-%m-%Y')
    
    # Formatea la columna 'Fecha'
    df_reset['TIME Format'] = df_reset['TIME'].apply(lambda x: format_date(x, 'EEE dd-MM-yyyy', locale=Locale('es', 'ES')))
    df_reset.to_csv(ruta_del_csv, mode='a', header=False,index=False)
def transformar_name(valor):
    str(valor)
    if valor.isupper():
        return valor
    else:
        palabras = valor.split()
        return ' '.join([palabras[0], palabras[2]])
def process_PAP(value):
    if any(map(str.isdigit, str(value))):
        value_float = int(value)
        value_str = str(value_float)
        return value_str
    else:
        return value
def load_df_by_name(directorios:str, cadena:str) -> pd.DataFrame:
    df_list = []
    for dictory in directorios:
        ruta = buscar_archivo_mas_antiguo(dictory, cadena)
        try:
            df = pd.read_excel(ruta, sheet_name='Hoja2')
            #print('Machea con nombre de hoja2')
        except Exception as e:
            print(f"Error: {e}. Trying with 'Sheet1'")
            df = pd.read_excel(ruta, sheet_name='Sheet1')
            #print('Machea con nombre de Sheet1')
        print(ruta)
        df_list.append(df)
    return df_list
def buscar_archivo_mas_antiguo(directorio, cadena):
    archivos_coincidentes = [os.path.join(directorio, nombre_archivo) for nombre_archivo in os.listdir(directorio) if cadena in nombre_archivo]
    if not archivos_coincidentes:
        return None
    else:
        archivo_mas_antiguo = min(archivos_coincidentes, key=os.path.getmtime)
        return archivo_mas_antiguo
def transformar_nombre(nombre):
    nombre = nombre.upper()
    nombre = nombre.replace('_', ' ')
    nombre = nombre.replace('-', ' ')
    return nombre

contrata_dic = {'DELTA ELECTRONICS (PERU) INC. S.R.L ELTEK PERU S.R.L.':'DELTA',
                'COMUNICACION FUTURA SOCIEDAD ANONIM':'COMFUTURA'}
columns_dic = {"CONTRA.FEC":"Fecha Contrata" , "CONTRAT.COM": "Comentario Contrata"
               ,"Dif Soles":"Diferencia Soles", "DOC COMPRAS": "Orden de Compra" 
               , "DOCUMENTO REFERENCIA": "DOCUMENTO REFERENCIA","NOMPROVEEDOR" :"CONTRATISTA"
               , "PAP":"N° PAP", "PEND FACT SOLES":"PENDIENTE EN SOLES"
               , "TEXTO BREVE": "DESCRIPCION"
               , "POSIC":"POSICIÓN","DESCRIPCION": "DESCRIPCIÓN"
               , "ANT PROYECTADO": "Antiguamiento Proyectado", "ANTIGUAMIENTO PAP" : "ANTIGUAMIENTO PAP"
               , "FECHA REPORTE": "FECHA REPORTE", "MES COMPROMISO" : "MES DE COMPROMISO"
               , "POS": "POSICION", 'FECH CONTAB':'FECHA CONTABILIDAD'}
def clean_date(df,column):
    df.loc[:,column] = pd.to_datetime(df[column])
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
    df[column] = df[column].astype(str)
    return df
# Ordeno los KEYS
def change_rows(df_concat:pd.DataFrame,df_duplicados:pd.DataFrame,Prepa_N:pd.DataFrame):
    df_concat = df_concat.sort_values(by=['CONCA', 'DOCUMENTO_REFERENCIA'], ascending=[True, True])
    # Crear un DataFrame con las keys y las columna que me interesa.
    df_info = df_duplicados.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='first')[['CONCA', 'DOCUMENTO_REFERENCIA', 'FECHASCOMPROMISO']]
    
    #la base en la que voy a editar las columnas,borro esa columna
    df_dupli = df_duplicados.drop('FECHASCOMPROMISO', axis=1).drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='last')
    df_final = merge_dataframes(df_dupli,df_info,['CONCA','DOCUMENTO_REFERENCIA'],index=df_dupli.index) #concateno, es como si intercambiara columnas
    df_final
    return Prepa_N.update(df_final,overwrite=True)#Finalmente con eso sobreescribro el original 


def drop_duplicatesR(df_duplicados:pd.DataFrame,BDF:pd.DataFrame,Prepa_N:pd.DataFrame):
    df2Rem = df_duplicados.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'],keep='first')
    list2Rem = df2Rem.index.tolist()
    BDF_no_D = BDF.drop(list2Rem,axis=0)
    Prepa_N = pd.concat([BDF_no_D, Prepa_N]) # este es un temporal para comprobar si hay filas duplicadas 
    print(f"los siguientes index en el BDF estan en el reporte aun: {list2Rem}")
    Prepa_N = Prepa_N.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'])
    return Prepa_N
def merge_dataframes(df1, df2, merge_on, index):
    
    merged_df = pd.merge(df1, df2, on=merge_on, how='left')
    display(len(merged_df))
    #display('Filas duplicadas:',merged_df[merged_df.duplicated(subset=['DOCUMENTO_REFERENCIA','CONCA'])]) 
    #return merged_df
    #merged_df = merged_df.drop_duplicates()
    merged_df = merged_df.drop_duplicates(subset=['CONCA','DOCUMENTO_REFERENCIA'],keep='first')
    #return merged_df
    print("filas del merge: ",len(merged_df))
    print("filas index: ",len(index))
    merged_df.set_index(index, inplace=True)
    return merged_df
def normalize_company_names(df, column):
    """Normaliza los nombres de las empresas en la columna especificada del DataFrame."""
    # Reemplaza "SAC" o "S. A. C." al final de los nombres de las empresas con "S.A.C"
    df[column] = df[column].str.replace(r"(SAC|S\. ?A\. ?C\.)$", "S.A.C.", regex=True)
    return df
def compact_rows(df:pd.DataFrame, columns:list, delimiter:str ='/'):
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    # Usar 'join' como función de agregación para concatenar los valores
    agg_func = {col: lambda x: '/'.join(x.unique()) for col in df.columns if col not in  columns}
    
    # Agrupar por 'OC Posición' y aplicar la función de agregación
    df1 = df.groupby(columns).agg(agg_func).reset_index()
    return df1
def update_and_rename(df1, update_cols, new_names):
    df = df1.copy()
    for col in update_cols:
        df[col[0]].update(df[col[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in update_cols for col in sublist[1:]], inplace=True)
    return df
def data_date(path:str):
    modification_time = os.path.getmtime(path)
    dt = datetime.fromtimestamp(modification_time)  
    dt = dt.date()
    # Formatea la fecha en el formato deseado
    return dt
def combine_and_rename(df1, combine_cols, new_names):
    df = df1.copy()
    for cols in combine_cols:
        df[cols[0]] = df[cols[0]].combine_first(df[cols[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in combine_cols for col in sublist[1:]], inplace=True)
    return df
def limpio_duplicados(filas_in,filas_out):
    if filas_in <= filas_out: #Corrigo las filas duplicadas//
        with open(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\list.pkl', "rb") as f:
            Colums_2Act = pickle.load(f)
    else:
        with open(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\list_V2.pkl', "rb") as f:
            Colums_2Act = pickle.load(f)
    return Colums_2Act
    
lista_columns_standar = ['DOC_COMPRAS','POSIC','CONCA','DOCUMENTO_REFERENCIA','FONDO','TIPO_MAT','TEXTO_BREVE','CODPROVEEDOR','NOMPROVEEDOR','FECH_CONTAB',
                         'MON','INDICADOR','EA_EM','FACTURADO','PEND_FACT','PEND_FACT_SOLES','ESTATUS_ACEPT','FECH_LIB_AF','DEMORA','ANTIGUAMIENTO','RESPONSABLE',
                         'RESPONSABLE2','CE_GESTOR','SUB_DIRECCION','FECHA_REPORTE','SOLICITUD','ESTATUS_SOLICITUD','AREA_RESPONSABLE','RESPONSABLE_PAP','ACCION','ANTIGUAMIENTO_PAP',
                         'ELEMENTO_PEP','FECHAS COMPROMISO','MES - COMPROMISO','DÍAS -2','ANT-PROYECTADO']
def get_recent_df_B(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente sin guion bajo creado de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    # Filtra los archivos que no contienen "_" en su nombre
    archivos_sin_guion_bajo = [archivo for archivo in archivos if "_" not in os.path.basename(archivo)]
    archivo_mas_reciente = max(archivos_sin_guion_bajo, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N

def get_near_time_path(path2find:str,path2search:str,kind2search:str,time_jump:int):
    Fecha_creacion2find = os.path.getctime(glob.glob(path2find)[0])
    Fecha_N_before= Fecha_creacion2find - time_jump #2Meses
    print(f"Busco archivo cercano a:{time.ctime(Fecha_N_before)}")
    archivos = glob.glob(path2search+"*"+kind2search)
    date_archivos = list(map(lambda x: os.path.getctime(x),archivos)) ## lista de times
    resultado = numero_mas_cercano(date_archivos,Fecha_N_before) # busco el time mas close
    ruta_EA = archivos[int(date_archivos.index(resultado))] # con ese valor obtengo la ruta 
    print(ruta_EA)
    return ruta_EA
def numero_mas_cercano(lista, objetivo):
    return min(lista, key=lambda x: abs(x - objetivo))    
def pre_proces(df: pd.DataFrame ,columns_2str: list[str] ,column_filter: str,C_format) -> pd.DataFrame: 
    """Anotaciones del tipo de cada parametro para hacer la funcin mas descriptiva"""
    ## Preprosecing of PREP_NEW
    df = convert_columns(df.copy(),columns_2str,C_format ) #Convert to str a key column
    df_EI = df.loc[df[column_filter] == 'Eduardo Iberico']#Filter
    df_EI = df_EI.copy()  # Crea una copia del DataFrame original para evitar modificar los datos originales
    return df_EI
def convert_columns(df, columns,type):
    for column in columns:
        df[column] = df[column].astype(type)
    return df    

def formated2bi(df_diff: pd.DataFrame, contrata_dic: dict, columns_dic: dict, colum_list: list):
    df_formated = df_diff.copy()
    
    # Normalizo nombres de columna
    df_formated.columns = map(transformar_nombre, df_formated.columns)
    df_formated['SITE'] = df_formated['SITE'].str.replace('_', ' ').str.title()
    df_formated['TEXTO BREVE'] = df_formated['TEXTO BREVE'].str.title()
    df_formated['NOMPROVEEDOR'].replace(contrata_dic, inplace=True)
    df_formated['NOMPROVEEDOR'] = df_formated['NOMPROVEEDOR'].str.title()
    df_formated['MES COMPROMISO'].replace('Setiembre', 'Septiembre', inplace=True)
    df_formated.columns = df_formated.columns.str.strip()
    
    # Cambio los nombres de las columnas con un Map
    df_formated.rename(columns=columns_dic, inplace=True)
    df_formated.columns = df_formated.columns.str.title()
    df_formated = standar_columns(df_formated,column_list) # Seguro anti falta de columnas 

    return df_formated

def standar_columns(df:pd.DataFrame,columns:list[str]):
    try:
        df1 = df[columns].copy()
    except KeyError as e:
        # Identificamos las columnas faltantes
        missing_columns = list(set(columns) - set(df.columns))
        # Rellenamos las columnas faltantes con valores vacíos
        for col in missing_columns:
            df.loc[:,col] = ''
        # Seleccionamos las columnas nuevamente
        df1 = df[columns].copy()   
    return df1  
with open(r'D:\Scripts1\Code\ActPEA\CODE\Temps\ListaColumnsBI.pkl', "rb") as archivo: # Columnas estadar para el PBI
    column_list = pickle.load(archivo)

    
BDF_dntdo = [   450066161920,
                450066161930,
                450066161910,
                
                450066025560,
                4500660255150,
                4500660255140,
                450066025540,
                450066025550,
                450063536120,
                450063536130,
                450063536110,
    
                450067094910,
                4500660086160,
                4500660087290,
                4500660087470,
                450065154150]

Hoy es: 29.08


# Cargo Archivos

## Cargo Prepa_old

In [138]:
PrePa_O,_ = get_recent_c_df(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa','Sheet1') 
#PrePa_O,_ = get_recent_df(r'D:\Prepa_local','Sheet1')
## Manejo los comentarios por si tiene fechas 
PrePa_O['COMENTARIO'] = PrePa_O['COMENTARIO'].apply(convert_to_date)
#CPX['FECHA'] = CPX['FECHA'].apply(convert_to_date)
PrePa_O = PrePa_O.dropna(subset=['CONCA'])



\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa\PREPA_UPDATE29.08.xlsx


## OTS

In [139]:
OT,_= get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\OTs','Hoja2')
OT = OT[OT['Status OT'] != 'PDTE CONTRATA'] #	
OT = OT[OT['Status OT'] != 'PDTE RESPONSABLE'].copy()
OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'],dayfirst=True).dt.date
OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'], format='%d-%m-%Y')
fecha_limite = pd.Timestamp('2021-11-01')
OT_cut = OT[OT['Fecha de Creación'] > fecha_limite].copy() # Corto las OTs solo de este año 
OT_cut = normalize_company_names(OT_cut,'Contrata')


D:\Scripts1\Code\ActPEA\archvis\OTs\OTS 21.08.2024.xlsx


C:\Users\C26764\AppData\Local\Temp\ipykernel_27928\3491332154.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'],dayfirst=True).dt.date


## Cargo Prepa_N

In [140]:
from datetime import datetime
#Carpeta de informes base
Prepa_N,path_archivo = get_recent_df('D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW/','DATA') 
Prepa_N = cleanrows(Prepa_N)
# Obtén la última fecha de modificación del archivo
data_time = data_date(path_archivo)
a = Today_date-data_time
if data_time == Today_date:
    reporte_old = False
    print("El reporte es actual")
elif data_time < Today_date:
    report_old = True
    print(f"El reporte es de hace {a.days} dias")

D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW\Prepasivo completo al 27.08.24.xlsx
El reporte es de hace 2 dias


## Cargo EAs_old 4 responsables 

1. Fecha de cracion del prepa Actual
2. Resto la fecha menos 2 meses
3. Cargo la ruta de los archivos y obtengo la lista de tiempos de todos los archivos
4. Encuentro la fecha mas cercana
5. devuelve la ruta del EA mas cercano
6. Cargo el archivo 

In [141]:
ruta_EA = get_near_time_path(path_archivo,r"C:\Users\C26764\America Movil Peru S.A.C\EAS - 1/",
                             ".xlsx",5260032)
                             

Busco archivo cercano a:Thu Jun 27 12:51:59 2024
C:\Users\C26764\America Movil Peru S.A.C\EAS - 1\EAUPDATE25.06.xlsx


In [142]:
EA_data = pd.read_excel(ruta_EA)
EA_data_cut = EA_data[["CONCATENADO","RESPONSABLE_DE_EA"]]
dicc_respo = EA_data_cut.set_index('CONCATENADO')['RESPONSABLE_DE_EA'].to_dict()

In [143]:
EA_data_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CONCATENADO        1267 non-null   int64 
 1   RESPONSABLE_DE_EA  1253 non-null   object
dtypes: int64(1), object(1)
memory usage: 19.9+ KB


## Cargo PAP

In [144]:
PAP_O,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP','Hoja2')# PAP de Otros
#PAP_O,_ = get_recent_df_B(r'D:\Scripts1\Code\ActPEA\archvis\PAP','Hoja2')# PAP de Otros

PAP_A,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin
#PAP_A,_ = get_recent_df_B(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin

PAP = pd.concat([PAP_O,PAP_A])

D:\Scripts1\Code\ActPEA\archvis\PAP\PAP 29.08_2.xlsx
D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo\PAP A 29.08.2024_2.xlsx


In [145]:

columns_E = lista_columns_standar
columns_Ac = Prepa_N.columns.tolist() 
if columns_Ac != lista_columns_standar:
    Columns_diff = list(set(columns_Ac) - set(columns_E))
    respuesta = input(f'Hay una variacion de columnas en la Base,{Columns_diff} \n¿Deseas continuar con la ejecución? (s/n): ')
    if respuesta.lower() != 's':
            sys.exit()
          

Hay una variacion de columnas en la Base,['CENFILE', 'CODIGO_SITE', 'ESTADO_CENFILE', 'INDICADOR2', 'FECHA - PROY', 'STATUS FINAL', 'NOMBRE_SITE'] 
¿Deseas continuar con la ejecución? (s/n):  S


## Cargo los Bloqueos de Factura

In [146]:
BDF = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\BAF_4FC.csv')


In [147]:
if 'STATUSFINAL' in Prepa_N.columns:
    BDF['STATUSFINAL'] == 'PENDIENTE'
    deep_columns = BDF[['MES - COMPROMISO','STATUS FINAL','FECHASCOMPROMISO']]  

elif 'STATUS FINAL' in Prepa_N.columns:
    BDF['STATUS FINAL'] == 'PENDIENTE'
    deep_columns = BDF[['MES - COMPROMISO','STATUS FINAL','FECHASCOMPROMISO']]  
else:
    deep_columns = BDF[['MES - COMPROMISO','FECHASCOMPROMISO']]  

BDF.drop_duplicates(subset=['CONCA','DOCUMENTO_REFERENCIA'],inplace=True)
BDF.drop([16],axis=0,inplace=True) # duplicado
# Selecciona las primeras 10 columnas
first_ten = BDF.iloc[:, :8]

# Combina las columnas seleccionadas en un nuevo DataFrame
BDF_cut = pd.concat([first_ten, deep_columns], axis=1)

In [148]:
BDF_cut.dropna(subset='CONCA',inplace=True)
BDF_cut.CONCA = BDF_cut.CONCA.astype('int64')
BDF_cut.rename(columns={'FECHASCOMPROMISO':'FECHAS COMPROMISO'},inplace=True)
BDF_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 112
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DOC_COMPRAS           112 non-null    float64
 1   POSIC                 112 non-null    float64
 2   CONCA                 112 non-null    int64  
 3   DOCUMENTO_REFERENCIA  112 non-null    float64
 4   TEXTO_BREVE           112 non-null    object 
 5   PEND_FACT_SOLES       112 non-null    float64
 6   NOMPROVEEDOR          112 non-null    object 
 7   RESPONSABLE2          112 non-null    object 
 8   MES - COMPROMISO      112 non-null    object 
 9   STATUS FINAL          112 non-null    object 
 10  FECHAS COMPROMISO     112 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 10.5+ KB


## Compruebo las filas que voy a insertar
- La variable reporte_old(boolean) indica si el reporte de prepa es nuevo o antiguo
- La variable row_e indica si las filas del bloque existen en el PREPA_N

In [149]:
# Concatenar los dos DataFrames
df_concat_0 = pd.concat([BDF_cut, Prepa_N]) # este es un temporal para comprobar si hay filas duplicadas 
# Crear una copia del DataFrame concatenado
df_copia = df_concat_0.copy()
# Eliminar las filas duplicadas basándose en las columnas de identificación
df_final_0 = df_concat_0.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='last')
# Obtener las filas duplicadas
df_duplicados = df_copia[df_copia.duplicated(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep=False)] ## Estas son las duplicas, 
#si este no es vacio entonces mi variable True Para duplicados
row_e = not(df_duplicados.empty)
df_duplicados.PEND_FACT_SOLES.sum()

0.0

In [150]:
if (row_e == True) and (report_old==True):
    print("Las filas existen en el Prepa_N, solo cambio datos")
    Prepa_N = change_rows(df_concat_0,df_duplicados,Prepa_N)
elif (row_e == True) and (report_old==False):
    Prepa_N = drop_duplicatesR(df_duplicados,BDF_cut,Prepa_N)
else: 
    print("Las filas no existen en el reporte, concateno")
    Prepa_N = pd.concat([Prepa_N, BDF_cut]) 
    Prepa_N.reset_index(drop=True,inplace=True)


Las filas no existen en el reporte, concateno


In [151]:
Prepa_N['MES - COMPROMISO'].value_counts()

MES - COMPROMISO
Octubre          330
Agosto           312
Setiembre        305
Noviembre        268
Bloqueo de AF    112
Diciembre         34
Mayo               8
Nuevas EA          5
Julio              3
Name: count, dtype: int64

# Prepocesing

## Preprosecing of OTs

In [152]:
Proyectos = ['ROLLOUT - 2023','ROLLOUT - 2022','ROLLOUT - 2024','CANON - 2023',]
OT_cut_mant = OT_cut[OT_cut['Tipo Req'] == 'MANT. MEJORA DE RED'].copy()
OT_cut_M = OT_cut[OT_cut.Proyecto == 'EXPANSIÓN'].copy()
OT_cut_RRL = OT_cut[OT_cut.Etiqueta.isin(Proyectos)].copy()

OT_concat = pd.concat([OT_cut_M,OT_cut_RRL,OT_cut_mant],axis=0)
OT_concat.rename(columns={'Codigo de Site':'ID_SITIO',
                           'Contrata':'NOMPROVEEDOR',
                           'Nombre de Site':'SITE'},inplace=True)
OT_concat = OT_concat[['OT','ID_SITIO','SITE','Proyecto','NOMPROVEEDOR','Etiqueta','Status OT']].copy()

OT_agg_ID_PRO = compact_rows(OT_concat,['SITE','NOMPROVEEDOR'],'/')
#OT_agg_ID_NM_PRO = compact_rows(OT_cut_RRL_2M,['ID_SITIO','SITE','PROVEEDOR'],'/')

## Prepro of PAP

In [153]:
PAP_f = PAP[['OC Posición','N° Sol','Estado','Acción','Nombre responsable','Id.SIte',
             'SIte','# Días','F.Creación','F.Modifica']]
##Filtro solo del norte
PAP_f = PAP_f.dropna(subset=['Id.SIte'])
#PAP_f = PAP_f[PAP_f['# Días'] < 500]

PAP_f_N = PAP_f[PAP_f['Id.SIte'].str.startswith(('L','T','SAD','CL','CAC'))].copy()
PAP_f_N = PAP_f_N.rename(columns={'N° Sol': 'PAP', #Rename
                               'SIte': 'SITE',
                               'Estado': 'ESTADO_PAP',
                                '# Días': 'ANTIGUAMIENTO_PAP',
                                'Id.SIte': 'ID Site',
                             'Nombre responsable': 'RESPONSABLE_PAP' })
PAP_S = Ac.split_ocs(PAP_f_N) # Spliteo OCs


In [154]:
## Consuto si se quiere ejecutar la actualizaccion total o solo parcial 

In [155]:
a =  Act_PBI(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa',3,column_list)

['PREPA_UPDATE29.08.xlsx', 'PREPA_UPDATE27.08.xlsx']
Bi actualizado


In [156]:
def goto_section_2():
    print("Saltando a la sección PBI")
    Act_PBI(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa',3,column_list)

In [157]:
# Preguntar al usuario si quiere saltar a una sección específic
user_input = input("¿Quieres Actualizar de forma Completa, o Parcial?(C/P)").strip().lower()   
if user_input == 'p':
        goto_section_2()
        sys.exit()
else:pass

¿Quieres Actualizar de forma Completa, o Parcial?(C/P) C


## Preprosecing of PREP

In [158]:
############################################### PASO 01#####################################################################
Prepa_EI_NC = pre_proces(Prepa_N,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','int64')
#####################################################################################################
###############################################################################################3
Prepa_EI = pre_proces(Prepa_EI_NC,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','str')
                    
Prepa_EI.loc[:, "OC Posición"] = Prepa_EI.loc[:, "DOC_COMPRAS"].str.cat(Prepa_EI.loc[:, "POSIC"], sep= ":")
Monto_In = Prepa_EI["PEND_FACT_SOLES"].sum() #Mont of USD 
#Prepa_EI = Prepa_EI.drop(columns=['FECHAS COMPROMISO','MES - COMPROMISO'])
#Prepa_EI.DOCUMENTO_REFERENCIA = Prepa_EI.DOCUMENTO_REFERENCIA.astype(str)

In [159]:
len(Prepa_EI)

249

In [160]:
## Preprosecing of PREP_OLD
PrePa_O_EI = pre_proces(PrePa_O,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','str')
PrePa_O_EI.replace('0x2a', np.nan, inplace=True)
PrePa_O_EI['PAP'] = PrePa_O_EI['PAP'].replace('sin pap', np.nan)
PrePa_O_EI = PrePa_O_EI.rename(columns={'FECHA': 'F.Creación'}) #Rename
#Filter of PEA_old
PrePa_O_EI = PrePa_O_EI[['DOCUMENTO_REFERENCIA','PAP','CONCA', 'SITE', 'COMENTARIO','ESTADO', 'RESPONSABLE3','CONTRA.FEC','CONTRAT.COM','ANTIGUAMIENTO_PAP']] #Columns i need 4 heredar
PrePa_O_EI.CONCA = PrePa_O_EI.CONCA.astype('int64')
Prepa_EI.CONCA = Prepa_EI.CONCA.astype('int64')
Prepa_EI = Prepa_EI.drop_duplicates(subset=['DOCUMENTO_REFERENCIA','CONCA'],keep='first')
len(PrePa_O_EI)

249

In [161]:
PrePa_O_EI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249 entries, 6 to 1376
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DOCUMENTO_REFERENCIA  249 non-null    int64  
 1   PAP                   249 non-null    float64
 2   CONCA                 249 non-null    int64  
 3   SITE                  249 non-null    object 
 4   COMENTARIO            249 non-null    object 
 5   ESTADO                249 non-null    object 
 6   RESPONSABLE3          232 non-null    object 
 7   CONTRA.FEC            18 non-null     object 
 8   CONTRAT.COM           47 non-null     object 
 9   ANTIGUAMIENTO_PAP     249 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 21.4+ KB


# Paso 02 (heredo info)

In [162]:
## ########################### # Merge new con el old (PASO 02) ####################################################################################
#####################################################################################################################3
PrePa_act = merge_dataframes(Prepa_EI,PrePa_O_EI,['CONCA','DOCUMENTO_REFERENCIA'], index=Prepa_EI.index)### Info heredada del old REDY 

#Mantengo el index para luego
###### Creo columnas para el que actualice el PAP####################### ///
PrePa_act[['F.Creación','Acción','RESPONSABLE_PAP','F.Modifica']] = pd.NA
PrePa_act.rename(columns={'ESTADO' : 'ESTADO_PAP'},inplace=True)
PrePa_act['RESPONSABLE_PAP'] =PrePa_act['RESPONSABLE_PAP'].astype(str)
try:
    PrePa_act = combine_and_rename(PrePa_act,[('ANTIGUAMIENTO_PAP_x', 'ANTIGUAMIENTO_PAP_y')],{'ANTIGUAMIENTO_PAP_x':'ANTIGUAMIENTO_PAP'})
except: pass
Cash_in = PrePa_act.PEND_FACT_SOLES.sum() 


249

filas del merge:  249
filas index:  249


In [164]:
PrePa_act.ESTADO_PAP.value_counts()

ESTADO_PAP
Finalizado              80
Pre registro            66
CHOCOS                  37
Observado               30
Solic FAC               13
Registrar Pendientes    10
Edwin                    5
JULIO A.                 2
MIGUEL R.                2
Marco                    1
Wilbert                  1
ORLANDO Z.               1
PAC                      1
Name: count, dtype: int64

In [165]:
PrePa_act.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249 entries, 6 to 1376
Data columns (total 54 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DOC_COMPRAS           249 non-null    object        
 1   POSIC                 249 non-null    object        
 2   CONCA                 249 non-null    int64         
 3   DOCUMENTO_REFERENCIA  249 non-null    float64       
 4   FONDO                 137 non-null    object        
 5   TIPO_MAT              137 non-null    object        
 6   TEXTO_BREVE           249 non-null    object        
 7   CODPROVEEDOR          137 non-null    float64       
 8   NOMPROVEEDOR          249 non-null    object        
 9   FECH_CONTAB           137 non-null    datetime64[ns]
 10  MON                   137 non-null    object        
 11  INDICADOR             137 non-null    object        
 12  EA_EM                 137 non-null    float64       
 13  FACTURADO             0 

# Paso 03

In [166]:
Filas_limpias = limpio_duplicados(filas_in,filas_out)######
PRE_all_act =  Ac.update_values_optimized_V2(PrePa_act, PAP_S, "OC Posición",Filas_limpias) 

In [167]:
PRE_all_act.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249 entries, 6 to 1376
Data columns (total 54 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DOC_COMPRAS           249 non-null    object        
 1   POSIC                 249 non-null    object        
 2   CONCA                 249 non-null    int64         
 3   DOCUMENTO_REFERENCIA  249 non-null    float64       
 4   FONDO                 137 non-null    object        
 5   TIPO_MAT              137 non-null    object        
 6   TEXTO_BREVE           249 non-null    object        
 7   CODPROVEEDOR          137 non-null    float64       
 8   NOMPROVEEDOR          249 non-null    object        
 9   FECH_CONTAB           137 non-null    datetime64[ns]
 10  MON                   137 non-null    object        
 11  INDICADOR             137 non-null    object        
 12  EA_EM                 137 non-null    float64       
 13  FACTURADO             0 

In [168]:

#PRE_all_act.drop(columns = 'OC Posición',inplace=True)
display(PRE_all_act.PEND_FACT_SOLES.sum())


duplicated_index = PRE_all_act.index.duplicated(keep=False)
df_duplicated = PRE_all_act[duplicated_index].copy()
PRE_all_act.drop(PRE_all_act[duplicated_index].index, inplace=True) # Borro duplicados del principal para tratarlos en el otro datafra 

# Ordena el DataFrame por la columna de fechas  LA FECHA DE CREACION ME SIRVE PARA ELMINAR DUPLICADOS

df_duplicated['F.Modifica'] = pd.to_datetime(df_duplicated['F.Modifica'], format="%d/%m/%Y %I:%M:%S %p") # Paso a fecha para ordenar
df_duplicated.sort_values('F.Modifica',ascending=False, inplace=True)
df_duplicated_NR = df_duplicated[df_duplicated.ESTADO_PAP != 'Rechazado'].sort_index(ascending=False)

df_duplicated_NR = df_duplicated_NR.loc[~df_duplicated_NR.index.duplicated(keep='first')] # niego la condicional que me selecciona cada indice duplicado excepto el primero
#df_duplicated['F.Creación'] = df_duplicated['F.Creación'].dt.strftime("%m/%d/%Y %I:%M:%S %p") # Regreso a str 
PRE_all_act = pd.concat([PRE_all_act, df_duplicated_NR])
PRE_all_act = cleanrows(PRE_all_act)

#PRE_all_act = PRE_all_act.drop(columns=['F.Modifica'])
Cash_out = PRE_all_act.PEND_FACT_SOLES.sum()
print(Cash_in - Cash_out)
#PRE_all_act['ESTADO_PAP'] = PRE_all_act['ESTADO_PAP'].fillna('Finalizado')
PRE_all_act.ESTADO_PAP.value_counts(dropna=False)

2955110.74

0.0


ESTADO_PAP
Finalizado          80
Pre registro        66
Aprobaciones FAC    47
Observado           31
FAC                 13
Visita ejecutada    10
Aprobaciones PAC     1
PAC                  1
Name: count, dtype: int64

In [169]:
PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].replace('<NA>','SIN RESPONSABLE')
# Aplicar la transformación
PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].apply(transformar_name)

In [170]:
PRE_all_act.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249 entries, 6 to 1376
Data columns (total 54 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DOC_COMPRAS           249 non-null    object        
 1   POSIC                 249 non-null    object        
 2   CONCA                 249 non-null    int64         
 3   DOCUMENTO_REFERENCIA  249 non-null    float64       
 4   FONDO                 137 non-null    object        
 5   TIPO_MAT              137 non-null    object        
 6   TEXTO_BREVE           249 non-null    object        
 7   CODPROVEEDOR          137 non-null    float64       
 8   NOMPROVEEDOR          249 non-null    object        
 9   FECH_CONTAB           137 non-null    datetime64[ns]
 10  MON                   137 non-null    object        
 11  INDICADOR             137 non-null    object        
 12  EA_EM                 137 non-null    float64       
 13  FACTURADO             0 

### Mapeo Compuesto (Tratamiento de Estados)

In [171]:
# Definimos los valores que estamos buscando en una lista
valores_buscados = ['Registrado', 'Aprobaciones FAC', 'Aprobaciones PAC','Lista Pendientes']

# Creamos la nueva columna basada en la condición
PRE_all_act['ESTADO'] = np.where(PRE_all_act['ESTADO_PAP'].isin(valores_buscados), 
                                 PRE_all_act['RESPONSABLE_PAP'], 
                                 PRE_all_act['ESTADO_PAP'])
map_estado = {'Auto ATP' : 'Registrar Pendientes',
       'Visita ejecutada' : 'Registrar Pendientes',
        'Programado' : 'WILBERT ATP',
        'FAC' : 'Solic FAC',
        'Gustavo Chocos':'CHOCOS',
        'Edwin Hurtado':'Edwin',
        'Marco Mejia':'Marco',
        'Wilbert Pintado':'Wilbert',
        'Orlando Zapata':'ORLANDO Z.',
        'Miguel Rojas':'MIGUEL R.',
        'Julio Arciniega':'JULIO A.',
        'Juan Trigo':'JUAN JOSE',
        'Eduardo Iberico':'Eduardo I.',
            }
Estados_finales = [ "Miguel Rojas",
                    "Finalizado",
                    "Orlando Z.",
                    "Chocos",
                    "Miguel R.",
                    "Julio A.",
                    "Marco",
                    "Edwin"]

PRE_all_act['ESTADO'] = PRE_all_act['ESTADO'].replace(map_estado)

PRE_all_act['RESPONSABLE3'] = PRE_all_act['RESPONSABLE3'].fillna(PRE_all_act['CONCA'].map(dicc_respo))

#Modifico el valor par adetemrina condicion
PRE_all_act.loc[PRE_all_act['ESTADO'] == 'SIN RESPONSABLE', 'ESTADO'] = 'Por Asignar'
PRE_all_act.loc[PRE_all_act['ESTADO'].isin(Estados_finales), 'COMENTARIO'] = ' '

a = PRE_all_act.ESTADO.value_counts(dropna=False).reset_index()
a.to_csv(f'D:/Prepa_R/Recuento_estados-{Today_D_M}.csv',index_label=False)
display(a)

,ESTADO,count
0,Finalizado,80
1,Pre registro,66
2,Observado,31
3,CHOCOS,25
4,Solic FAC,13
5,ORLANDO Z.,13
6,Registrar Pendientes,10
7,MIGUEL R.,6
8,JULIO A.,3
9,Eduardo I.,1


In [172]:
PRE_all_act = PRE_all_act.drop(columns=['F.Creación','RESPONSABLE_PAP'
                                        ,'ANTIGUAMIENTO','DEMORA','FECH_LIB_AF','INDICADOR','MON'])



In [173]:
PRE_all_act['MES - COMPROMISO'].replace('Setiembre','Septiembre',inplace=True)

In [174]:
display(PRE_all_act[PRE_all_act.ESTADO.isna()])


,DOC_COMPRAS,POSIC,CONCA,DOCUMENTO_REFERENCIA,FONDO,TIPO_MAT,TEXTO_BREVE,CODPROVEEDOR,NOMPROVEEDOR,FECH_CONTAB,...,PAP,SITE,COMENTARIO,ESTADO_PAP,RESPONSABLE3,CONTRA.FEC,CONTRAT.COM,Acción,F.Modifica,ESTADO


# Enbullo la info actualizada del norte en el global
    04. Para ello necesito primero darle formato al global

In [175]:
Resultado = PRE_all_act
columnas_para_actualizar = ['ANTIGUAMIENTO_PAP', 'PAP','SITE','COMENTARIO','RESPONSABLE3','CONTRA.FEC','CONTRAT.COM','ESTADO_PAP','ESTADO']
Resultado_reducido = Resultado[columnas_para_actualizar]
#############################################################
Prepa_N = Prepa_N.dropna(axis=1, how='all')
unhavent_columns = set(Resultado.columns.tolist()) - set(Prepa_N.columns.tolist()) # Columnas que le faltan al grande del chico
# Crear las columnas necesarias con dtype 'object'
for col in list(unhavent_columns):
    Prepa_N[col] = pd.Series(dtype='object')
## Df pequeño en el Df grande 
Resultado.columns = Resultado.columns.str.replace(' ', '')
Prepa_N.columns = Prepa_N.columns.str.replace(' ', '')
Prepa_N.update(Resultado_reducido, overwrite=True) ### Aqui es dond eembullo la data.
Prepa_N.columns = Prepa_N.columns.astype(str)
Prepa_N_1 = Ac.convert_columns_to_str(Prepa_N.copy(), ['COMENTARIO'])
Prepa_N_1 = Prepa_N_1.drop(columns=['ESTADO_PAP','Acción','SOLICITUD','ESTATUS_SOLICITUD','AREA_RESPONSABLE','ACCION','TIPO_MAT','CE_GESTOR'])
Prepa_N_1 = Prepa_N_1.replace({'nan':''})
cols = list(Prepa_N_1.columns)
Prepa_N_1 = Prepa_N_1[cols[:-9] + ['ANTIGUAMIENTO_PAP','PAP', 'SITE', 'ESTADO','COMENTARIO','RESPONSABLE3','CONTRA.FEC','CONTRAT.COM']]
Prepa_N_1.COMENTARIO = Prepa_N_1.COMENTARIO.replace({'1899-12-30 00:00:00' : ''})
Prepa_N_1 = Prepa_N_1.dropna(axis=1, how='all') # elimino columnas vacias
Prepa_N_1 = Prepa_N_1.loc[:,~Prepa_N_1.columns.duplicated()]

# 2 Excel

In [176]:
import datetime


In [177]:
#
useless_columns= ['DOCUMENTO_REFERENCIA','FONDO','FECH_CONTAB','MON','INDICADOR',
                      'PEND_FACT','EA_EM','ESTATUS_ACEPT','RESPONSABLE','DEMORA',
                  'SUB_DIRECCION','FECHA_REPORTE','ELEMENTO_PEP','CENFILE','ESTADO_CENFILE','CODIGO_SITE','INDICADOR2',
                      'ÁREA','AGRUPADOR','RESPON','NOMBRE SITE','RESPONSABLE_PAP']
## Formato Tabla en el excel 
#Today_D_M=Today_D_M+'_2'
Ac.Excel_format(Prepa_N_1,r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa\PREPA_UPDATE{}.xlsx'.format(Today_D_M),useless_columns)
#Ac.Excel_format(Prepa_N_1,r'D:\Prepa_local\PREPA_UPDATE{}.xlsx'.format(Today_D_M),useless_columns)

filename = r'D:\Scripts1\Code\ActPEA\CODE\last_run.json'
# Carga la última fecha de ejecución
last_run_date = Ac.load_last_run_date(filename)

# Comprueba si la celda ya se ha ejecutado hoy
if last_run_date != datetime.datetime.now().date():
    # Tu código aquí
    print('Ejecutado: ',Today_str)
    process_to_bcsv(PRE_all_act,'D:/Prepa/TIME.S/Prepa_TS1.csv',Today_str)
    process_to_bcsv(PRE_all_act,r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Prepa_TS.csv',Today_str)# añado al acumulado

    # Guarda la fecha de hoy como la última fecha de ejecución
    Ac.save_last_run_date(filename)
else:
    print("El código ya se ha ejecutado hoy.")


El código ya se ha ejecutado hoy.


# To BI

In [134]:
def Act_PBI(Path:str,range_diff,column_list):
        
    ## Revisar esto porque se ejecuto 2 veces el mismo dia pero en diferentes idles
    dataframes = get_2_df_from_path(Path,'.xlsx',range_diff,'Sheet1')
    #dataframes = get_2_df_from_path(r'D:\Prepa_local','.xlsx',4,'Sheet1')
    # Ruta de donde extraigo los archivos a comparar,intervalo,Nombre de la hoja
    dataframes_filt = list(map(filtro,dataframes)) # filtro solo del norte
    df_diff = calc_diff_V1(dataframes_filt[0],dataframes_filt[1]) # Funcion que me crea la columna de diferencia.
    df_diff = cleanrows(df_diff)
    #Divide la plata para el agg del BI 
    df_diff['DIFERENCIA EN SOLES'] = df_diff['DIFERENCIA EN SOLES'].transform(lambda x: x / df_diff['DIFERENCIA EN SOLES'].value_counts()[x] if pd.notnull(x) else x) 
    
    df_diff = normalize_company_names(df_diff,'NOMPROVEEDOR') 
    
    df_diff_m = pd.merge(df_diff,OT_agg_ID_PRO[['SITE','NOMPROVEEDOR','Status OT']],on=['SITE','NOMPROVEEDOR'],how='left')
    
    df_diff_m.loc[df_diff_m.CONCA.isin(BDF_dntdo),'Status OT'] = 'No Ejecutado'#BDF No ejecutados por lista
    
    try:
        df_diff_m.PAP = df_diff_m.PAP.astype('int')
        df_diff_m.drop(columns=['ANT-PROYECTADO'],inplace=True)
    except:pass
    df_diff_m.rename(columns={'Status OT':'ANTIGUAMIENTO PROYECTADO'},inplace = True)
    ### esta pasando un df vacio , probablemente falta la columna o los valores
    df_fomarted = formated2bi(df_diff_m,contrata_dic,columns_dic,column_list)
    
    if 'Estado Cenfile' in df_fomarted.columns:
        df_fomarted['Estado Cenfile'] = '0'
    else: pass
    #return df_fomarted
    df_fomarted.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Reporte Prepasivo.csv',index=False)
    subprocess.call(["python",r"D:\Scripts1\Code\ActPEA\CODE\2BI_Norma.py"])
    print('Bi actualizado')
    

In [178]:
Act_PBI(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa',3,column_list)

['PREPA_UPDATE29.08.xlsx', 'PREPA_UPDATE27.08.xlsx']
Bi actualizado


In [180]:
print(df_fomarted['Mes De Compromiso'].value_counts())

NameError: name 'df_fomarted' is not defined

0